In [2]:
import requests 
import pandas as pd 
import json
from dateutil import parser

In [3]:
API_KEY = "c41c5f98704c8106dabb4483fef2932e-75e8bd9303e1044e1368557f7ac8114a"
ACCOUNT_ID = "101-012-24490475-001"
OANDA_URL = "https://api-fxpractice.oanda.com/v3"

In [4]:
session = requests.Session()

In [5]:
session.headers.update({
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
})

In [6]:
params = dict(
    count=10,
    granularity= "D",
    price = "MBA"
)

In [7]:
url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments"

In [8]:
response = session.get(url, params=params, data=None, headers=None) 

In [9]:
response.status_code

200

In [10]:
data = response.json()

In [11]:
instruments_list = data['instruments']


In [12]:
len(instruments_list)


128

In [13]:
instruments_list[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'minimumGuaranteedStopLossDistance', 'guaranteedStopLossOrderExecutionPremium', 'guaranteedStopLossOrderLevelRestriction', 'tags', 'financing'])

In [14]:
key_i=['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision','marginRate' ]

In [15]:
instruments_dict = {}
for i in instruments_list:
    key = i['name']
    instruments_dict[key] = {k:i[k] for k in key_i}

In [16]:
instruments_dict['EUR_USD']

{'name': 'EUR_USD',
 'type': 'CURRENCY',
 'displayName': 'EUR/USD',
 'pipLocation': -4,
 'displayPrecision': 5,
 'tradeUnitsPrecision': 0,
 'marginRate': '0.0333'}

In [17]:
with open("../data/instruments.json" , "w")as f:
    f.write(json.dumps(instruments_dict, indent=2))

In [18]:
def fetch_candles(pair_name, count=10,granularity="H1"):
    url = f"{OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
    count= count,
    granularity= granularity,
    price = "MBA"
    )
    response = session.get(url, params=params, data=None, headers=None) 
    data =response.json()

    if response.status_code == 200:
        if 'candles'not in data:
            data=[]
        else:
            data = data['candles']
        return response.status_code, data   
    
def get_candles_df(data):
    if len(data)== 0:
        return pd.DataFram()
    
    prices= ['mid', 'bid', 'ask']
    ohlc= ['o','h','l','c']

    final_data =[]
    for candle in data:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = parser.parse(candle['time'])
        new_dict['volume'] = candle['volume']
        for p in prices :
            for o in ohlc:
                new_dict[f"{p}_{o}"] = float(candle[p][o])
            
        final_data.append(new_dict)
    df = pd.DataFrame.from_dict(final_data)   
    return df
    
def create_data_file(pair_name, count=10,granularity="H1"):
    code, data =fetch_candles(pair_name ,count, granularity)
    if code != 200:
        print("Failed",pair_name,data)  
        return                       
    if len(data)== 0:
          print("No candles",pair_name)  
             
    candles_df  = get_candles_df(data)  
    candles_df.to_pickle(f"../data/{pair_name}_{granularity}.pkl") 
    print(f"{pair_name} {granularity}{candles_df.shape[0]} candles, {candles_df.time.min()} {candles_df.time.max()}")                         
                              
    

In [19]:
code, data =fetch_candles("EUR_USD" ,count=10, granularity="H4")
candles_df = get_candles_df(data)

In [20]:
create_data_file("EUR_USD" ,count=10, granularity="H1")


EUR_USD H49 candles, 2023-01-22 22:00:00+00:00 2023-01-24 06:00:00+00:00


In [22]:
data

[{'complete': True,
  'volume': 11607,
  'time': '2023-01-22T22:00:00.000000000Z',
  'bid': {'o': '1.08623', 'h': '1.08989', 'l': '1.08599', 'c': '1.08941'},
  'mid': {'o': '1.08650', 'h': '1.08998', 'l': '1.08617', 'c': '1.08948'},
  'ask': {'o': '1.08676', 'h': '1.09008', 'l': '1.08630', 'c': '1.08956'}},
 {'complete': True,
  'volume': 9596,
  'time': '2023-01-23T02:00:00.000000000Z',
  'bid': {'o': '1.08938', 'h': '1.09025', 'l': '1.08883', 'c': '1.08915'},
  'mid': {'o': '1.08946', 'h': '1.09033', 'l': '1.08892', 'c': '1.08923'},
  'ask': {'o': '1.08955', 'h': '1.09041', 'l': '1.08900', 'c': '1.08931'}},
 {'complete': True,
  'volume': 22966,
  'time': '2023-01-23T06:00:00.000000000Z',
  'bid': {'o': '1.08915', 'h': '1.09261', 'l': '1.08863', 'c': '1.09110'},
  'mid': {'o': '1.08922', 'h': '1.09270', 'l': '1.08872', 'c': '1.09118'},
  'ask': {'o': '1.08930', 'h': '1.09280', 'l': '1.08881', 'c': '1.09125'}},
 {'complete': True,
  'volume': 26392,
  'time': '2023-01-23T10:00:00.0000